In [171]:
import tensorflow as tf
from load_data import read_data_sets
from tensorflow.models.rnn import rnn_cell
from tensorflow.models.rnn import rnn

In [172]:
vocab_size = 10000
rel_vocab_size = 25

In [173]:
datasets = read_data_sets(datafolder='/home/rupchap/data/NYT/',
                          vocab_size=vocab_size, rel_vocab_size=rel_vocab_size,
                          validation_size=5000, test_size=500)

load pre-processed data


In [174]:
# get number of training examples from data
num_examples = datasets.train.num_examples

In [175]:
# graph params
# input_dimension = 1 vocab_size
predict_classes = rel_vocab_size
learning_rate = 0.01
training_epochs = 5
batch_size = 50
display_step = 5
embed_size = 30
hidden_size = 5
num_steps = 5

In [177]:
def embed(x):
    print 'x:', x
    # Embedding
    with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, embed_size])
        inputs = tf.nn.embedding_lookup(embedding, x)
    print 'inputs:', inputs
    return inputs

In [178]:
def inference(inputs):
        
    lstm_cell = rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0) 
    
    initial_state = lstm_cell.zero_state(batch_size, tf.float32)
    inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(1, num_steps, inputs)]
    
    print 'inputs:', inputs
    
    outputs, state = rnn.rnn(lstm_cell, inputs, initial_state)
    output = tf.reshape(tf.concat(1, outputs), [-1, hidden_size])
    print 'output:', output
    softmax_w = tf.get_variable("softmax_w", [hidden_size, predict_classes])
    softmax_b = tf.get_variable("softmax_b", [predict_classes])
    logits = tf.matmul(output, softmax_w) + softmax_b
    print 'logits:', logits
    return logits

In [179]:
def loss(logits, y):
    dot_product = y * tf.log(logits)
    y = tf.to_int64(y)
    xentropy = -tf.reduce_sum(dot_product, reduction_indices=1)
    loss = tf.reduce_mean(xentropy)
    return loss

In [180]:
def training(cost, global_step):
    tf.scalar_summary('cost', cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [181]:
def evaluate(output, y):
    correct_prediction = tf.equal(tf.arg_max(output, dimension=1),
                                  tf.arg_max(y, dimension=1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [182]:
with tf.Graph().as_default():
    
    # TODO: will currently pad to max sentence length in training
    # Consider modifying to use bucketed models.
    max_sentence_length = max([len(sentence) for sentence in datasets.train.sentences])
    
    x = tf.placeholder(dtype='int32', shape=[None, max_sentence_length])
    y = tf.placeholder(dtype='int32', shape=[None, predict_classes])
    
    print 'num steps:', num_steps
    inputs = embed(x)
    logits = inference(inputs)
    cost = loss(logits, y)
    
    global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
    
    train_op = training(cost, global_step)
    
    eval_op = evaluate(output, y)
    
    summary_op = tf.merge_all_summaries()
    
    sess = tf.Session()
    
    summary_writer = tf.train.SummaryWriter('tflogs/', graph_def=sess.graph_def)
    
    init_op = tf.initialize_all_variables()
    
    sess.run(init_op)
    
    # Training loop over epochs
    for epoch in range(training_epochs):
        avg_cost = 0.
        
        num_batches = int(num_examples/batch_size)
        # Loop over batches
        for i in range(num_batches):
            batch_x, batch_y = datasets.train.next_batch(batch_size)
            
            # pad sentences up to length of longest sentence in batch
            # ToDo: fix this to use buckets instead for efficiency
            PAD_ID = 1
            for sentence in batch_x:
                sentence += [PAD_ID] * (max_sentence_length - len(sentence))
            print batch_x
            
            # train on batch data
            feed_dict = {x: batch_x, y: batch_y}
            sess.run(train_op, feed_dict=feed_dict)
            
            # compute average loss
            minibatch_cost = sess.run(cost, feed_dict=feed_dict)
            avg_cost += minibatch_cost/num_batches
            
            # display logs
            summary_str = sess.run(summary_op, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, global_step=sess.run(global_step))
                       
        print 'Optimisation complete'

num steps: 5
x: Tensor("Placeholder:0", shape=(?, 104), dtype=int32)
inputs: Tensor("embedding_lookup:0", shape=(?, 104, 30), dtype=float32, device=/device:CPU:0)


ValueError: Number of ways to split should evenly divide the split dimension but got split_dim 1 (size = 104) and num_split 5